In [1]:
from importlib import import_module
from snntoolbox.parsing.utils import *
from snntoolbox.parsing.model_libs.keras_input_lib import *
from snntoolbox.bin.utils import update_setup, import_target_sim
from snntoolbox.conversion.utils import normalize_parameters
from snntoolbox.datasets.utils import get_dataset
from snntoolbox.simulation.utils import *
from snntoolbox.simulation.target_simulators.brian2_target_sim import *

Using TensorFlow backend.
/home/wolin/.conda/envs/tf2/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/wolin/.conda/envs/tf2/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/wolin/.conda/envs/tf2/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/wolin/.conda/envs/tf2/lib/python3.7

In [2]:
def run(snn, **test_set):
            return snn.run(**test_set)

In [3]:
model_path = "/home/wolin/snn_toolbox/forgetting/full/"

In [4]:
config = update_setup(model_path + "config")







Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

SNN toolbox Warning: Currently, turning off Poisson input is
only possible in INI simulator. Falling back on Poisson input.


In [5]:
model = "keras_mnist"

In [6]:
n = load(model_path, model)

In [7]:
mp = ModelParser(n['model'], config)

In [8]:
mp.parse()
pm = mp.build_parsed_model()

Parsing layer Conv2D.
Using activation relu.
Parsing layer Conv2D.
Using activation relu.
Parsing layer MaxPooling2D.
Skipping layer Dropout.
Parsing layer Flatten.
Parsing layer Dense.
Using activation relu.
Skipping layer Dropout.
Parsing layer Dense.
Using activation softmax.

Building parsed model...

Compiling parsed model...



In [9]:
normset, testset = get_dataset(config)

Loading scale factors from disk instead of recalculating.
Loading data set from '.npz' files in /home/wolin/snn_toolbox/forgetting/full/data.



In [10]:
normalize_parameters(pm, config, **normset)

Normalizing parameters...
Using scale factor 1.00 for softmax layer.



In [11]:
target_sim = import_target_sim(config)

In [12]:
spiking_model = target_sim.SNN(config, None)

Initializing brian2 simulator...



In [13]:
spiking_model.build(pm)

WARNING    /home/wolin/.conda/envs/tf2/lib/python3.7/site-packages/keras/utils/conv_utils.py:93: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.copy(kernel[slices])
 [py.warnings]


Building spiking model...
Building layer: 00Conv2D_32x26x26
................................

WARNING    /home/wolin/snn_toolbox/snntoolbox/simulation/target_simulators/brian2_target_sim.py:369: RuntimeWarning: Biases not implemented.
  warnings.warn("Biases not implemented.", RuntimeWarning)
 [py.warnings]



Connecting layer...
Building layer: 01Conv2D_64x24x24
................................................................
Connecting layer...


WARNING    /home/wolin/snn_toolbox/snntoolbox/simulation/utils.py:1241: RuntimeWarning: Layer type 'MaxPooling' not supported yet. Falling back on 'AveragePooling'.
  "Falling back on 'AveragePooling'.", RuntimeWarning)
 [py.warnings]


Building layer: 02MaxPooling2D_64x12x12
................................................................
Building layer: 03Flatten_9216
Building layer: 04Dense_128
Lenght of weights:1179648
Building layer: 05Dense_10
Lenght of weights:1280
Compiling spiking model...

Detected layer with biases: 00Conv2D_32x26x26
Detected layer with biases: 01Conv2D_64x24x24
Detected layer with biases: 04Dense_128
Detected layer with biases: 05Dense_10
Number of operations of ANN: 24043530
Number of neurons: 67850
Number of synapses: 13903616



In [14]:
spiking_model.snn.objects

[PoissonGroup(784, rates=0. * hertz),
 Thresholder(clock=Clock(dt=1. * msecond, name='poissongroup_clock'), when=thresholds, order=0, name='poissongroup_thresholder'),
 NeuronGroup(clock=Clock(dt=1. * msecond, name='neurongroup_clock'), when=start, order=0, name='neurongroup'),
 Thresholder(clock=Clock(dt=1. * msecond, name='neurongroup_clock'), when=thresholds, order=0, name='neurongroup_thresholder'),
 Resetter(clock=Clock(dt=1. * msecond, name='neurongroup_clock'), when=resets, order=0, name='neurongroup_resetter'),
 StateUpdater(clock=Clock(dt=1. * msecond, name='neurongroup_clock'), when=groups, order=0, name='neurongroup_stateupdater'),
 NeuronGroup(clock=Clock(dt=1. * msecond, name='neurongroup_1_clock'), when=start, order=0, name='neurongroup_1'),
 Thresholder(clock=Clock(dt=1. * msecond, name='neurongroup_1_clock'), when=thresholds, order=0, name='neurongroup_1_thresholder'),
 Resetter(clock=Clock(dt=1. * msecond, name='neurongroup_1_clock'), when=resets, order=0, name='neuron

In [15]:
config.get('output', 'plot_vars')

'{}'

In [16]:
config.set('output', 'plot_vars', 'none')

In [17]:
config.get('output', 'plot_vars')

'none'

In [18]:
results = run(spiking_model, **testset)


Starting new simulation...

Current accuracy of batch:
Starting simulation at t=0. s for a duration of 50. ms
0.003 (6%) simulated in < 1s, estimated 1s remaining.
0.004 (8%) simulated in < 1s, estimated 1s remaining.
0.005 (10%) simulated in < 1s, estimated 1s remaining.
0.006 (12%) simulated in < 1s, estimated 1s remaining.
0.006999999999999999 (13%) simulated in < 1s, estimated 1s remaining.
0.008 (16%) simulated in < 1s, estimated 1s remaining.
0.009000000000000001 (18%) simulated in < 1s, estimated 1s remaining.
0.01 (20%) simulated in < 1s, estimated 1s remaining.
0.011 (21%) simulated in < 1s, estimated 1s remaining.
0.012 (24%) simulated in < 1s, estimated 1s remaining.
0.013000000000000001 (26%) simulated in < 1s, estimated 1s remaining.
0.013999999999999999 (27%) simulated in < 1s, estimated 1s remaining.
0.015 (30%) simulated in < 1s, estimated 1s remaining.
0.016 (32%) simulated in < 1s, estimated 1s remaining.
0.017 (34%) simulated in < 1s, estimated 1s remaining.
0.01800

0.038 (75%) simulated in < 1s, estimated < 1s remaining.
0.039 (77%) simulated in < 1s, estimated < 1s remaining.
0.04 (80%) simulated in < 1s, estimated < 1s remaining.
0.041 (82%) simulated in < 1s, estimated < 1s remaining.
0.042 (84%) simulated in < 1s, estimated < 1s remaining.
0.043000000000000003 (86%) simulated in < 1s, estimated < 1s remaining.
0.044 (87%) simulated in < 1s, estimated < 1s remaining.
0.045 (89%) simulated in < 1s, estimated < 1s remaining.
0.046 (92%) simulated in < 1s, estimated < 1s remaining.
0.047 (94%) simulated in < 1s, estimated < 1s remaining.
0.048 (96%) simulated in < 1s, estimated < 1s remaining.
0.049 (98%) simulated in < 1s, estimated < 1s remaining.
0.05 (100%) simulated in < 1s

Batch 3 of 10 completed (30.0%)
Moving accuracy of SNN (top-1, top-1): 100.00%, 100.00%.
Moving accuracy of ANN (top-1, top-1): 100.00%, 100.00%.

Resetting simulator...

Starting new simulation...

Current accuracy of batch:
Starting simulation at t=0. s for a duration 

0.023 (46%) simulated in < 1s, estimated < 1s remaining.
0.024 (48%) simulated in < 1s, estimated < 1s remaining.
0.025 (50%) simulated in < 1s, estimated < 1s remaining.
0.026000000000000002 (52%) simulated in < 1s, estimated < 1s remaining.
0.026999999999999996 (53%) simulated in < 1s, estimated < 1s remaining.
0.027999999999999997 (55%) simulated in < 1s, estimated < 1s remaining.
0.028999999999999998 (57%) simulated in < 1s, estimated < 1s remaining.
0.03 (60%) simulated in < 1s, estimated < 1s remaining.
0.031 (62%) simulated in < 1s, estimated < 1s remaining.
0.032 (64%) simulated in < 1s, estimated < 1s remaining.
0.033 (66%) simulated in < 1s, estimated < 1s remaining.
0.034 (68%) simulated in < 1s, estimated < 1s remaining.
0.035 (70%) simulated in < 1s, estimated < 1s remaining.
0.036000000000000004 (72%) simulated in < 1s, estimated < 1s remaining.
0.037 (73%) simulated in < 1s, estimated < 1s remaining.
0.038 (75%) simulated in < 1s, estimated < 1s remaining.
0.039 (77%) si

0.006999999999999999 (13%) simulated in < 1s, estimated 2s remaining.
0.008 (16%) simulated in < 1s, estimated 2s remaining.
0.009000000000000001 (18%) simulated in < 1s, estimated 2s remaining.
0.01 (20%) simulated in < 1s, estimated 2s remaining.
0.011 (21%) simulated in < 1s, estimated 2s remaining.
0.012 (24%) simulated in < 1s, estimated 2s remaining.
0.013000000000000001 (26%) simulated in < 1s, estimated 2s remaining.
0.013999999999999999 (27%) simulated in < 1s, estimated 2s remaining.
0.015 (30%) simulated in < 1s, estimated 2s remaining.
0.016 (32%) simulated in < 1s, estimated 2s remaining.
0.017 (34%) simulated in < 1s, estimated 2s remaining.
0.018000000000000002 (36%) simulated in < 1s, estimated 1s remaining.
0.019 (37%) simulated in < 1s, estimated 1s remaining.
0.02 (40%) simulated in < 1s, estimated 1s remaining.
0.021 (42%) simulated in 1s, estimated 1s remaining.
0.022 (43%) simulated in 1s, estimated 1s remaining.
0.023 (46%) simulated in 1s, estimated 1s remaining

WARNING    /home/wolin/snn_toolbox/snntoolbox/simulation/utils.py:745: RuntimeWarning: invalid value encountered in true_divide
  avg_acc = np.mean(np.true_divide(match, count))
 [py.warnings]


Simulation finished.


Total accuracy: 100.00% on 10 test samples.


Accuracy averaged over classes: nan%
